# 熱中症患者数予測
#### 気象データから熱中症患者数を予測する回帰モデルを作成する

### 03.deploy.ipynb
Webサービスへのデプロイ・推論

In [1]:
import azureml
from azureml.core import Workspace, Run

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.57


## 1. スコアリングスクリプトの作成(score.py)

In [2]:
%%writefile score.py
import json
import numpy as np
import pandas as pd
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model
import azureml.train.automl

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('heatstroke_model')
    model = joblib.load(model_path)
def run(raw_data):
    data = pd.read_json(json.loads(raw_data)['data'])
    y_hat = model.predict(data)   
    
    return y_hat.tolist()

Overwriting score.py


## 2. Dockerイメージの準備

### Docker用環境ファイルの作成

In [3]:
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies.create(
    conda_packages=['numpy','pandas','scikit-learn'],
    pip_packages=['azureml-sdk[automl]'])

conda_env_file_name = 'myenv.yml'
myenv.save_to_file('.', conda_env_file_name)

with open("myenv.yml", "w") as f:
    f.write(myenv.serialize_to_string())

### 環境ファイルの中身確認

In [4]:
with open("myenv.yml", "r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-sdk[automl]==1.0.57.*
- numpy
- pandas
- scikit-learn
channels:
- conda-forge



### Dockerイメージを作成

In [5]:
from azureml.core.model import Model
ws = Workspace.from_config()
model = Model(ws, "heatstroke_model")
print(model.name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FW4LXLQYW to authenticate.
Interactive authentication successfully completed.
heatstroke_model


↓こちらの実行完了には、少しお時間かかります。

In [6]:
%%time
from azureml.core.image import Image, ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

image = Image.create(name = "heatstroke-image",
                              models = [model], # this is the registered model object
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running.............................................................................................
Succeeded
Image creation operation finished for image heatstroke-image:4, operation "Succeeded"
CPU times: user 793 ms, sys: 726 ms, total: 1.52 s
Wall time: 8min 20s


## 3. Azure Container Instance(ACI)へのデプロイ

### ACI用環境ファイルの作成

In [7]:
from azureml.core.webservice import AciWebservice, Webservice

# 構成
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               memory_gb=8,
                                               tags={"data": "heatstroke",
                                                     "method": "automl"},
                                               description='Heatstroke patient prediction Model')

### DockerイメージをACIにデプロイ
↓こちらの実行完了には、少しお時間かかります。
※nameの値は、イメージ名であり、既に作っている名前と同じ名前は作ることができません。

In [9]:
%%time
service = Webservice.deploy_from_image(workspace=ws,
                                       name='automl-heatstroke-prediction00',
                                       deployment_config=aciconfig,
                                       image=image)

service.wait_for_deployment(show_output=True)
print(service.state)

Creating service
Running...........................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy
CPU times: user 319 ms, sys: 141 ms, total: 460 ms
Wall time: 2min 29s


### WebサービスのURLを取得

↓実行後、出力されたURLにアクセスしてみてください。

In [10]:
print(service.scoring_uri)

http://1e012f3a-ea73-4513-81c2-e357c60b5a8d.westus.azurecontainer.io/score


## 4. 生の HTTP 要求を送信して Web サービスをテスト

### 2019年のデータを用いて、搬送人員（計）を予測

In [1]:
import requests
import pandas as pd
import io
import json

df_test = pd.read_csv('./data/Heatstroke_patient_prediction_predict_data.csv')

X_score = df_test.drop(["年月日","年","月","搬送人員（計）"], axis=1)
test_samples1 = json.dumps({"data":  X_score.to_json()})
headers = {'Content-Type': 'application/json'}
resp_str = requests.post(service.scoring_uri, test_samples1, headers=headers)

resp_json = json.dumps({"data":  resp_str.text})

df_purpose = pd.read_json(json.loads(resp_json)['data'])
df_purpose.columns = ["搬送人員（計）"]
df_purpose

NameError: name 'service' is not defined

## 5. ローカルディレクトリ・Azure Blob Storageへ推論結果を格納

### 説明変数と推論結果をファイルに書き出し

In [ ]:
df_nonTrans = df_test.drop(["搬送人員（計）"], axis=1)

df_true = pd.concat([df_nonTrans,df_purpose],axis=1)
df_true
data_path = "./data/Heatstroke_patient_prediction_output.csv"
df.to_csv(data_path)

### Azure Blob Storageに格納

In [ ]:
ds = ws.get_default_datastore()
print(f"{ds.name}\n{ds.datastore_type}\n{ds.account_name}\n{ds.container_name}\n{ds.account_key}")
ds.upload_files([data_path],overwrite=True)

## 5. リソースのクリーンアップ

In [ ]:
service.delete()